<a href="https://colab.research.google.com/github/rohithbommala/Pytorch-Tutorial/blob/main/Dlfa_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Configuring the path of json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d jeffheaton/count-the-paperclips

100% 1.65G/1.65G [00:20<00:00, 112MB/s] 
100% 1.65G/1.65G [00:20<00:00, 88.1MB/s]


In [ ]:
from zipfile import ZipFile
dataset = '/content/count-the-paperclips.zip'

In [ ]:
with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import os
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from PIL import Image

class CustomDataLoader(Dataset):
    def __init__(self, train_image_ids, train_csv, images_folder):
        self.train_image_ids = train_image_ids
        self.train_csv = train_csv
        self.images_folder = images_folder
        self.mean = torch.tensor([0.485, 0.456, 0.406])
        self.std = torch.tensor([0.229, 0.224, 0.225])
        self.transform = transforms.Compose([
            transforms.Resize((3, 28, 28)),
            transforms.RandomRotation(10),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std)
        ])

    def __len__(self):
        return len(self.train_image_ids)

    def __getitem__(self, idx):
        image_id = self.train_image_ids[idx]
        image_path = os.path.join(self.images_folder, f'{image_id}.jpg')
        image = Image.open(image_path).convert('RGB')
        label = self.train_csv.loc[self.train_csv['image_id'] == image_id, 'count'].values[0]

        # Apply random rotation augmentation
        rotation_angle = torch.randint(-int(str(idx)[-2:]), int(str(idx)[-2:]), (1,)).item()
        self.transform.transforms.insert(1, transforms.RandomRotation(rotation_angle))

        # Apply random horizontal flipping
        flip_prob = int(str(idx)[-2:]) / 100
        self.transform.transforms.insert(2, transforms.RandomHorizontalFlip(p=flip_prob))

        transformed_image = self.transform(image)
        flattened_image = transformed_image.view(-1)

        return image_id, flattened_image, label

In [ ]:
train_csv = pd.read_csv('train.csv')
train_image_ids = train_csv['id']
images_folder = '/content/clips-data-2020/clips'

In [ ]:
data_loader = CustomDataLoader(train_image_ids, train_csv, images_folder)

ValueError: If size is a sequence, it should have 1 or 2 values